## Sentiment Analysis & Opinion Mining

### Spelling Correction

pip install pytypo

Other spelling correction libraries - "topy", "hunspell" and "autocorrect".

In [1]:
sentence = 'This iPhone thng isss realllllllllly coooolll.......'

import pytypo
sentenceX = pytypo.correct_sentence(sentence)
print(sentenceX)

This iPhone thing is really cool.


## Sentiment analysis using SentiWordNet

In [2]:
import nltk
from nltk.stem import *
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn

## Convert from PennTreebank tags to Wordnet tags
def pennTOwn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

## My Sentiment Analysis Logic
def mySAL(sentence):
    pscore = 0
    nscore = 0
    oscore = 0
    wordcount = 0
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)

    print(tagged)
    
    for tag in tagged:
        
        lemma = ''
        pos = pennTOwn(tag[1])
    
        if (pos != None):
            lemma = lemmatizer.lemmatize(tag[0], pos = pos)
            wordcount+= 1
           
        if 'VB' in tag[1] and len(list(swn.senti_synsets(lemma,'v'))) > 0:
            tempSynsets = list(swn.senti_synsets(lemma,'v'))
            P = tempSynsets[0].pos_score()
            N = tempSynsets[0].neg_score()
            O = tempSynsets[0].obj_score()
            pscore+= P
            nscore+= N
            oscore+= O
            output = lemma + " -- P: " + str(P) + ", N: " + str(N) + ", O: " + str(O)
            print(output)
        
        elif 'JJ' in tag[1] and len(list(swn.senti_synsets(lemma,'a'))) > 0:
            tempSynsets = list(swn.senti_synsets(lemma,'a'))
            P = tempSynsets[0].pos_score()
            N = tempSynsets[0].neg_score()
            O = tempSynsets[0].obj_score()
            pscore+= P
            nscore+= N
            oscore+= O
            output = lemma + " -- P: " + str(P) + ", N: " + str(N) + ", O: " + str(O)
            print(output)
        
        elif 'RB' in tag[1] and len(list(swn.senti_synsets(lemma,'r'))) > 0:
            tempSynsets = list(swn.senti_synsets(lemma,'r'))
            P = tempSynsets[0].pos_score()
            N = tempSynsets[0].neg_score()
            O = tempSynsets[0].obj_score()
            pscore+= P
            nscore+= N
            oscore+= O
            output = lemma + " -- P: " + str(P) + ", N: " + str(N) + ", O: " + str(O)
            print(output)
            
    print("Positive:", pscore, " Negative:", nscore, " Objectivity:", oscore)
    return pscore, nscore, oscore, wordcount

In [3]:
sentence = "I love eating this wonderful dish."
P, N, O, C = mySAL(sentence)

[('I', 'PRP'), ('love', 'VBP'), ('eating', 'VBG'), ('this', 'DT'), ('wonderful', 'JJ'), ('dish', 'NN'), ('.', '.')]
love -- P: 0.5, N: 0.0, O: 0.5
eat -- P: 0.0, N: 0.0, O: 1.0
wonderful -- P: 0.75, N: 0.0, O: 0.25
Positive: 1.25  Negative: 0.0  Objectivity: 1.75


Full list of TreeBank Part of Speech tags.

https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [4]:
nltk.help.upenn_tagset('VBP')
nltk.help.upenn_tagset('VBG')
nltk.help.upenn_tagset('JJ')

VBP: verb, present tense, not 3rd person singular
    predominate wrap resort sue twist spill cure lengthen brush terminate
    appear tend stray glisten obtain comprise detest tease attract
    emphasize mold postpone sever return wag ...
VBG: verb, present participle or gerund
    telegraphing stirring focusing angering judging stalling lactating
    hankerin' alleging veering capping approaching traveling besieging
    encrypting interrupting erasing wincing ...
JJ: adjective or numeral, ordinal
    third ill-mannered pre-war regrettable oiled calamitous first separable
    ectoplasmic battery-powered participatory fourth still-to-be-named
    multilingual multi-disciplinary ...


In [5]:
sentence = "I do not like this dish."
P, N, O, C = mySAL(sentence)

[('I', 'PRP'), ('do', 'VBP'), ('not', 'RB'), ('like', 'IN'), ('this', 'DT'), ('dish', 'NN'), ('.', '.')]
do -- P: 0.0, N: 0.0, O: 1.0
not -- P: 0.0, N: 0.625, O: 0.375
Positive: 0.0  Negative: 0.625  Objectivity: 1.375


In [6]:
sentence = "I don't like this dish."
P, N, O, C = mySAL(sentence)

[('I', 'PRP'), ('do', 'VBP'), ("n't", 'RB'), ('like', 'VB'), ('this', 'DT'), ('dish', 'NN'), ('.', '.')]
do -- P: 0.0, N: 0.0, O: 1.0
like -- P: 0.125, N: 0.0, O: 0.875
Positive: 0.125  Negative: 0.0  Objectivity: 1.875


##### WikiPedia List of Contractions

https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions

In [7]:
import re
contractions_dict = { 
"can't": "cannot",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"shan't": "shall not",
"sha'n't": "shall not",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"wasn't": "was not",
"wouldn't've": "would not have",
"you're": "you are",
"you've": "you have"
}

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(sentence):
    def replace(match):
        return contractions_dict[match.group(0)]

    return contractions_re.sub(replace, sentence)

In [8]:
sentence = "I don't like this dish."
sentenceX = expand_contractions(sentence)
P, N, O, C = mySAL(sentenceX)

[('I', 'PRP'), ('do', 'VBP'), ('not', 'RB'), ('like', 'IN'), ('this', 'DT'), ('dish', 'NN'), ('.', '.')]
do -- P: 0.0, N: 0.0, O: 1.0
not -- P: 0.0, N: 0.625, O: 0.375
Positive: 0.0  Negative: 0.625  Objectivity: 1.375


#### Scoring formulas:

1) Absolute Proportional Difference. Bounds: [0,1]
    Sentiment = (P − N) / (Word Count)

    Disadvantage: A sentence's score is affected by non-sentiment-related content.

2) Relative Proportional Difference. Bounds: [-1, 1]

    Sentiment = (P − N) / (P + N)

    Disadvantage: A sentence's score may tend to cluster very strongly near the scale endpoints (because they may contain content primarily or exclusively of either positive or negative).

3) Logit scale. Bounds: [-infinity, +infinity]

    Sentiment = log(P + 0.5) - log(N + 0.5)
    
    This tends to have the smoothest properties and is symmetric around zero. The 0.5 is a smoother to prevent log(0).

In [9]:
import numpy as np

def sentiScore(P, N, O, C):
    APD = RPD = LS = 0
    
    if (P + N + O) > 0:
        APD = (P - N) / C
        
    if (P + N) > 0:
        RPD = (P - N) / (P + N)
    
    LS = np.log10(P + 0.5) - np.log(N + 0.5)
    
    print("APD:", APD, " RPD:", RPD, " LS:", LS)
    
    return

In [10]:
sentence = "I hate the new phone."
sentenceX = expand_contractions(sentence)
P, N, O, C = mySAL(sentenceX)
sentiScore(P, N, O, C)

[('I', 'PRP'), ('hate', 'VBP'), ('the', 'DT'), ('new', 'JJ'), ('phone', 'NN'), ('.', '.')]
hate -- P: 0.0, N: 0.75, O: 0.25
new -- P: 0.375, N: 0.0, O: 0.625
Positive: 0.375  Negative: 0.75  Objectivity: 0.875
APD: -0.125  RPD: -0.3333333333333333  LS: -0.281135498292


In [11]:
sentence = "The price of the new iPhone is terrible."
sentenceX = expand_contractions(sentence)
P, N, O, C = mySAL(sentenceX)
sentiScore(P, N, O, C)

[('The', 'DT'), ('price', 'NN'), ('of', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('iPhone', 'NN'), ('is', 'VBZ'), ('terrible', 'JJ'), ('.', '.')]
new -- P: 0.375, N: 0.0, O: 0.625
be -- P: 0.25, N: 0.125, O: 0.625
terrible -- P: 0.0, N: 0.625, O: 0.375
Positive: 0.625  Negative: 0.75  Objectivity: 1.625
APD: -0.025  RPD: -0.09090909090909091  LS: -0.171991028867


## VADER - A rule-based model for sentiment analysis

http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf

Citation - Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

Note - "twython" Python package and NLTK add-ons needs to be installed.

NLTK add-ons can be installed by:

import nltk
nltk.download()


In [12]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

def myVader(sentence):
    sid = SIA()
    score = sid.polarity_scores(sentence)
    
    if (score['compound'] > 0):
        sentiment = "Positive"
    elif (score['compound'] < 0):
        sentiment = "Negative"
    else:
        sentiment = "Neutral"

    print("Overall Sentiment - ", sentiment)
    print("Sentiment Score - ", score)
    
    return

/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [13]:
sentence = """Potentially the worst written book I've read since 2005. You can learn python NLTK library IF 
you already know NLP concepts. You can't learn NLP from it."""

myVader(sentence)

Overall Sentiment -  Negative
Sentiment Score -  {'neg': 0.136, 'neu': 0.864, 'pos': 0.0, 'compound': -0.6249}


In [14]:
sentence = """This book is wonderfully written. It's full of interesting examples, and gradually and 
effortlessly introduces the reader to quite advanced topics in Natural Language Processing, Python, and 
machine learning. One of the few technical book that is worth reading cover-to-cover, just for 
the pleasure of it."""

myVader(sentence)

Overall Sentiment -  Positive
Sentiment Score -  {'neg': 0.0, 'neu': 0.702, 'pos': 0.298, 'compound': 0.9432}


In [15]:
sentence = "This is a book."
myVader(sentence)

Overall Sentiment -  Neutral
Sentiment Score -  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [16]:
sentence = ":-)"
myVader(sentence)

Overall Sentiment -  Positive
Sentiment Score -  {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.3182}


In [17]:
sentence = ":-("
myVader(sentence)

Overall Sentiment -  Negative
Sentiment Score -  {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.3612}


In [18]:
sentence = "LOL"
myVader(sentence)

Overall Sentiment -  Positive
Sentiment Score -  {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}


## Named Entity Recognizer (NER)

Citation: Jenny Rose Finkel, Trond Grenager, and Christopher Manning. 2005. Incorporating Non-local Information into Information Extraction Systems by Gibbs Sampling. Proceedings of the 43nd Annual Meeting of the Association for Computational Linguistics (ACL 2005), pp. 363-370. http://nlp.stanford.edu/~manning/papers/gibbscrf3.pdf

Stanford NER - https://nlp.stanford.edu/software/CRF-NER.shtml#Download

Accuracy:
https://pythonprogramming.net/testing-stanford-ner-taggers-for-accuracy/?completed=/named-entity-recognition-stanford-ner-tagger/

In [19]:
from nltk.tag.stanford import StanfordNERTagger
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.chunk import conlltags2tree
from nltk.tree import Tree
import os

java_path = 'C:/Program Files (x86)/Java/jre1.8.0_121/bin/java.exe'
os.environ['JAVAHOME'] = java_path
classifier = 'D:/Stanford/stanford-ner-2017-06-09/classifiers/english.all.3class.distsim.crf.ser.gz'
jar = 'D:/Stanford/stanford-ner-2017-06-09/stanford-ner.jar'

# Convert stanford NE to NLTK tree
def stanfordNE2BIO(tagged_sent):
    bio_tagged_sent = []
    prev_tag = "O"
    for token, tag in tagged_sent:
        if tag == "O": #O
            bio_tagged_sent.append((token, tag))
            prev_tag = tag
            continue
        if tag != "O" and prev_tag == "O": # Begin NE
            bio_tagged_sent.append((token, "B-"+tag))
            prev_tag = tag
        elif prev_tag != "O" and prev_tag == tag: # Inside NE
            bio_tagged_sent.append((token, "I-"+tag))
            prev_tag = tag
        elif prev_tag != "O" and prev_tag != tag: # Adjacent NE
            bio_tagged_sent.append((token, "B-"+tag))
            prev_tag = tag

    return bio_tagged_sent

# Convert stanford NE to NLTK tree
def stanfordNE2tree(ne_tagged_sent):
    bio_tagged_sent = stanfordNE2BIO(ne_tagged_sent)
    sent_tokens, sent_ne_tags = zip(*bio_tagged_sent)
    sent_pos_tags = [pos for token, pos in pos_tag(sent_tokens)]

    sent_conlltags = [(token, pos, ne) for token, pos, ne in zip(sent_tokens, sent_pos_tags, sent_ne_tags)]
    ne_tree = conlltags2tree(sent_conlltags)
    return ne_tree


def myNER(sentence):
    st = StanfordNERTagger(classifier, jar, encoding='utf-8')
    tokens = word_tokenize(sentence)
    classifiedTokens = st.tag(tokens)
    
    neTree = stanfordNE2tree(classifiedTokens)
    
    # To deal with situations like "Donald" and "Trump" is one Named Entity rather than two.
    neJoined = []
    for subtree in neTree:
        if type(subtree) == Tree: # If subtree is a noun chunk, i.e. NE != "O"
            ne_label = subtree.label()
            ne_string = " ".join([token for token, pos in subtree.leaves()])
            neJoined.append((ne_string, ne_label))
    
    neList = []
    
    for token in neJoined:
        if ((token[1] == 'PERSON') or (token[1] == 'ORGANIZATION') or (token[1] == 'LOCATION')):
            neList.append(token[0])
    
    ## Use dict to remove duplicates (Python 3.6 only)
    print("Named Entity List:", list(dict.fromkeys(neList)))
    return

In [20]:
sentence = """Neymar is now the highest salaried athlete on earth, but that doesn’t sound like his 
primary motivating factor in moving to Paris Saint-Germain. In a series of Instagram videos posted 
after the move, Neymar said that his father didn’t want him to leave Barcelona. He did it anyway because, 
in his words, “an athlete needs challenges.”"""

myNER(sentence)
myVader(sentence)

LookupError: Could not find stanford-ner.jar jar file at D:/Stanford/stanford-ner-2017-06-09/stanford-ner.jar

In [ ]:
sentence = """Holidaymakers across Europe face sweltering temperatures as a “dangerous” 
heatwave hits the continent. Scorching temperatures are forecast for countries such as Spain, 
Italy and Croatia with the mercury set to hit 40C prompting severe warnings."""

myNER(sentence)
myVader(sentence)

In [ ]:
sentence = """Michael Gove has confirmed that some foreign trawlers will still have access to UK 
waters after Brexit. Mr Gove, the UK environment secretary, said British fishermen would not have 
the capacity to land all of the fish in British territorial waters. And he said that some access would
therefore be granted to vessels from other countries. He was speaking during a fact-finding mission to 
Denmark, which was largely focused on the Danish food industry. The Danish fishing industry is currently 
highly-dependent on fish caught in UK territorial waters."""

myNER(sentence)
myVader(sentence)

## Extract main topics
Original code written by Shlomi Babluki
http://thetokenizer.com/2013/05/09/efficient-way-to-extract-the-main-topics-of-a-sentence/

Modified to use Stanford POS tagger instead of NLTK POS. And skip the topic items already picked up by our NER model above.
https://nlp.stanford.edu/software/tagger.shtml

Citations for Stanford POS tagger:
1) Kristina Toutanova and Christopher D. Manning. 2000. Enriching the Knowledge Sources Used in a Maximum Entropy Part-of-Speech Tagger. In Proceedings of the Joint SIGDAT Conference on Empirical Methods in Natural Language Processing and Very Large Corpora (EMNLP/VLC-2000), pp. 63-70.

2) Kristina Toutanova, Dan Klein, Christopher Manning, and Yoram Singer. 2003. Feature-Rich Part-of-Speech Tagging with a Cyclic Dependency Network. In Proceedings of HLT-NAACL 2003, pp. 252-259. 

In [ ]:
import nltk
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tokenize import word_tokenize

import os
java_path = 'C:/Program Files (x86)/Java/jre1.8.0_121/bin/java.exe'
os.environ['JAVAHOME'] = java_path

model = 'D:/Stanford/stanford-postagger-full-2017-06-09/models/english-bidirectional-distsim.tagger'
jarFile = 'D:/Stanford/stanford-postagger-full-2017-06-09/stanford-postagger.jar'

# This is the semi-CFG
##########################
cfg = {}
cfg["NNP+NNP"] = "NNP"
cfg["NN+NN"] = "NNI"
cfg["NNI+NN"] = "NNI"
cfg["JJ+JJ"] = "JJ"
cfg["JJ+NN"] = "NNI"
##########################


def topicExtractor(sentence):

    def normalize_tags(tagged):
        n_tagged = []
        for t in tagged:
            if t[1].endswith("S"):
                n_tagged.append((t[0], t[1][:-1]))
                continue
            n_tagged.append((t[0], t[1]))
        return n_tagged


    tokens = word_tokenize(sentence)
    myTagger = StanfordPOSTagger(model, jarFile)
    tags = normalize_tags(myTagger.tag(tokens))

    merge = True
    while merge:
        merge = False
        for x in range(0, len(tags) - 1):
            t1 = tags[x]
            t2 = tags[x + 1]
            key = "%s+%s" % (t1[1], t2[1])
            value = cfg.get(key, '')
            if value:
                merge = True
                tags.pop(x)
                tags.pop(x)
                match = "%s %s" % (t1[0], t2[0])
                pos = value
                tags.insert(x, (match, pos))
                break

    matches = []
    for t in tags:
        # Since our NER extractor already has the NNPs, we will only keep the NNIs
        # If you want to keep the NNPs, include t[1] == "NNP" in the if logic below.
        if t[1] == "NNI":
            matches.append(t[0])
        
        # Remove duplicates
        matches = list(dict.fromkeys(matches))
    
    print("Main Topic:", matches)
    return

In [ ]:
sentence = """Michael Gove has confirmed that some foreign trawlers will still have access to UK 
waters after Brexit. Mr Gove, the UK environment secretary, said British fishermen would not have 
the capacity to land all of the fish in British territorial waters. And he said that some access would
therefore be granted to vessels from other countries. He was speaking during a fact-finding mission to 
Denmark, which was largely focused on the Danish food industry. The Danish fishing industry is currently 
highly-dependent on fish caught in UK territorial waters."""

topicExtractor(sentence)
myNER(sentence)
myVader(sentence)

## Product aspect extraction

Extract noun or noun phrases.

In [ ]:
import nltk
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tokenize import sent_tokenize, word_tokenize

import os
java_path = 'C:/Program Files (x86)/Java/jre1.8.0_121/bin/java.exe'
os.environ['JAVAHOME'] = java_path

model = 'D:/Stanford/stanford-postagger-full-2017-06-09/models/english-bidirectional-distsim.tagger'
jarFile = 'D:/Stanford/stanford-postagger-full-2017-06-09/stanford-postagger.jar'

def aspectExtract(sentence):
    sentences = sent_tokenize(sentence)
    
    for sent in sentences:
        previousWord=''
        previousTag=''
        currentWord=''
        aspectList=[]

        #Extracting Aspects
        tokens = word_tokenize(sent)
        myTagger = StanfordPOSTagger(model, jarFile)
        tags = myTagger.tag(tokens)
        
        for word,tag in tags:
            if(tag == 'NN' or tag == 'NNP'):
                if(previousTag == 'NN' or previousTag == 'NNP'):
                    currentWord= previousWord + ' ' + word
                else:
                    if (previousWord != ''):
                        aspectList.append(previousWord)
                
                    currentWord = word

            previousWord = currentWord
            previousTag = tag
        
        aspectList.append(previousWord)
    
        print("Sentence -", sent)
        print("Aspect List:", aspectList)
        
        # How about we extract the sentiment too?
        myVader(sent)
           
    return

In [ ]:
sentence = "The battery life of an iPhone is fantastic. But the price is not very good."
aspectExtract(sentence)

### Decompose complex sentences
Idea is to do sentiment analysis on individual phrases as different parts of a complex sentence may have different sentiments.

Clause level Penn Treebank labels (S, SBAR, SBARQ, SQ).

http://www.surdeanu.info/mihai/teaching/ista555-fall13/readings/PennTreebankConstituents.html

In [ ]:
from nltk.parse.stanford import StanfordParser
from nltk.tokenize import sent_tokenize
from nltk.tree import *
import os

java_path = 'C:/Program Files (x86)/Java/jre1.8.0_121/bin/java.exe'
classPath = 'D:/Stanford/stanford-postagger-full-2017-06-09/stanford-postagger.jar:D:/Stanford/stanford-ner-2017-06-09/stanford-ner.jar:D:/Stanford/stanford-parser-full-2017-06-09/stanford-parser-3.8.0-models.jar:D:/Stanford/stanford-parser-full-2017-06-09/stanford-parser.jar'
models = 'D:/Stanford/stanford-postagger-full-2017-06-09/models:D:\Stanford\stanford-ner-2017-06-09\classifiers'

os.environ['JAVAHOME'] = java_path
os.environ['CLASSPATH'] = classPath
os.environ['STANFORD_MODELS'] = models

  
def traverseTree(tree):
    treeChunks = []
     
    for subtree in reversed(list(tree.subtrees())):
        if ((subtree.label() == 'S') or (subtree.label() == 'SBAR') or (subtree.label() == 'SBARQ') or (subtree.label() == 'SQ')):
            treeChunks.append(' '.join(subtree.leaves()))
            del tree[subtree.treeposition()]
                        
    # Remove any empty string
    if '' in treeChunks:
        treeChunks.remove('')
    
    return list(reversed(treeChunks))

def ClauseExtraction(sentence):
    myParser = StanfordParser('D:/Stanford/stanford-parser-full-2017-06-09/stanford-parser.jar', 'D:/Stanford/stanford-parser-full-2017-06-09/stanford-parser-3.8.0-models.jar', model_path="edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")

    sentences = sent_tokenize(sentence)
    newSentences = []
    
    for sent in sentences:
        tempSent = []
        tagged = next(myParser.raw_parse(sent))
        ptagged = ParentedTree.convert(tagged)

        print(traverseTree(ptagged))

    return    

In [ ]:
sentence = "The battery life of an iPhone is fantastic but the price is not very good"
ClauseExtraction(sentence)

In [ ]:
sentence = "ABC cites the fact that chemical additives are banned in many countries and feels they may be banned in this state too"
ClauseExtraction(sentence)

### Draw the tree structure of a POS tagged sentence for visual analysis

In [ ]:
from nltk.parse.stanford import StanfordParser
from nltk.tokenize import sent_tokenize
from nltk.tree import *
import os

java_path = 'C:/Program Files (x86)/Java/jre1.8.0_121/bin/java.exe'
classPath = 'D:/Stanford/stanford-postagger-full-2017-06-09/stanford-postagger.jar:D:/Stanford/stanford-ner-2017-06-09/stanford-ner.jar:D:/Stanford/stanford-parser-full-2017-06-09/stanford-parser-3.8.0-models.jar:D:/Stanford/stanford-parser-full-2017-06-09/stanford-parser.jar'
models = 'D:/Stanford/stanford-postagger-full-2017-06-09/models:D:\Stanford\stanford-ner-2017-06-09\classifiers'

os.environ['JAVAHOME'] = java_path
os.environ['CLASSPATH'] = classPath
os.environ['STANFORD_MODELS'] = models


def drawTree(sentence):
    myParser = StanfordParser('D:/Stanford/stanford-parser-full-2017-06-09/stanford-parser.jar', 'D:/Stanford/stanford-parser-full-2017-06-09/stanford-parser-3.8.0-models.jar', model_path="edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")

    sentences = sent_tokenize(sentence)
    newSentences = []
    
    for sent in sentences:
        tempSent = []
        tagged = next(myParser.raw_parse(sent))
        tagged.draw()

    return

In [ ]:
sentence = "ABC cites the fact that chemical additives are banned in many countries and feels they may be banned in this state too"
drawTree(sentence)